# Setup

In [1]:
import os

from rdflib import Namespace, Graph, SH
from rdflib.util import guess_format
from pyshacl import validate

import tasty.constants as tc
import tasty.graphs as tg
import tasty.entities as te
from tasty.shapes_loader import ShapesLoader


In [2]:
# turtle file name and path to save instance data
ttl_name = 'haystack_nrel_natural_gas_meter_data.ttl'
ttl_path = os.path.join(os.path.dirname(os.path.abspath('')), "tests/files/data/" + ttl_name)

# Create a namespace and load in a blank Brick / Haystack graph
SAMPLE = Namespace('urn:sample/')
hg = tg.get_versioned_graph(tc.HAYSTACK, tc.V3_9_10)
hg.bind('sample', SAMPLE)
h_ont = tg.load_ontology(tc.HAYSTACK, tc.V3_9_10)

# Specify the schema version (tc.V9_9_10, etc.) to use
hp = te.HaystackPointDefs(tc.V3_9_10)
he = te.HaystackEquipDefs(tc.V3_9_10)
hrefs = te.HaystackRefDefs(tc.V3_9_10)

# Bind all of the first class types as attributes
hp.bind()
he.bind()
hrefs.bind()

# Simple wrapper around all of the shapes
shrap = te.ShapesWrapper(tc.HAYSTACK, tc.V3_9_10)
shrap.bind()

# Haystack Equipment

In [3]:
# ID
equip_id = 'NREL-GAS-MTR-01'

# flow meter
flow_meter = he.flow_meter.deep_copy()
flow_meter.set_namespace(SAMPLE)
flow_meter.set_id(equip_id)

# thermal meter
thermal_meter = he.thermal_meter.deep_copy()
thermal_meter.set_namespace(SAMPLE)
thermal_meter.set_id(equip_id)

# bind to graph to use sync method
flow_meter.bind_to_graph(hg), thermal_meter.bind_to_graph(hg)

(True, True)

## Equipment Sensors

In [4]:
flow_rate_sensor = shrap.NaturalGasFlowRateShape.cast_to_entity(f"{equip_id}-NaturalGasFlowRateShape")
volume_sensor = shrap.NaturalGasVolumeShape.cast_to_entity(f"{equip_id}-NaturalGasVolumeShape")
energy_rate_sensor = shrap.NaturalGasEnergyRateShape.cast_to_entity(f"{equip_id}-NaturalGasEnergyRateShape")
energy_sensor = shrap.NaturalGasEnergyShape.cast_to_entity(f"{equip_id}-NaturalGasEnergyShape")

In [5]:
# lists
sensors = [[flow_rate_sensor, volume_sensor],
           [energy_rate_sensor, energy_sensor]]

# loop
for idx, list in enumerate(sensors):

    if idx == 0:
        meter = flow_meter
    elif idx == 1:
        meter = thermal_meter
        
    for sensor in list:
        sensor.set_namespace(SAMPLE)
        sensor.add_relationship(hrefs.equipRef, meter)
        sensor.sync()



Bound urn:sample/NREL-GAS-MTR-01-NaturalGasFlowRateShape to graph
Bound urn:sample/NREL-GAS-MTR-01-NaturalGasVolumeShape to graph
Bound urn:sample/NREL-GAS-MTR-01-NaturalGasEnergyRateShape to graph
Bound urn:sample/NREL-GAS-MTR-01-NaturalGasEnergyShape to graph


In [6]:
# print graph
ttl_data = hg.serialize(format='turtle').decode('utf-8')
print(ttl_data)

@prefix ph: <https://project-haystack.org/def/ph/3.9.10#> .
@prefix phIoT: <https://project-haystack.org/def/phIoT/3.9.10#> .
@prefix phScience: <https://project-haystack.org/def/phScience/3.9.10#> .
@prefix sample: <urn:sample/> .

sample:NREL-GAS-MTR-01-NaturalGasEnergyRateShape a phIoT:thermal-meter ;
    ph:hasTag phIoT:sensor,
        phScience:gas,
        phScience:power ;
    phIoT:equipRef sample:NREL-GAS-MTR-01 .

sample:NREL-GAS-MTR-01-NaturalGasEnergyShape a phIoT:thermal-meter ;
    ph:hasTag phIoT:sensor,
        phScience:energy,
        phScience:gas ;
    phIoT:equipRef sample:NREL-GAS-MTR-01 .

sample:NREL-GAS-MTR-01-NaturalGasFlowRateShape a phIoT:flow-meter ;
    ph:hasTag phIoT:sensor,
        phScience:flow,
        phScience:gas ;
    phIoT:equipRef sample:NREL-GAS-MTR-01 .

sample:NREL-GAS-MTR-01-NaturalGasVolumeShape a phIoT:flow-meter ;
    ph:hasTag phIoT:sensor,
        phScience:gas,
        phScience:volume ;
    phIoT:equipRef sample:NREL-GAS-MTR-01 .

sa

# Validate

In [7]:
sl = ShapesLoader(tc.HAYSTACK)
shapes_graph = sl.load_all_shapes()
shapes_graph.bind('sample', SAMPLE)

In [8]:
shapes_graph.add((tc.PH_SHAPES_NREL['NREL-Natural-Gas-Meter-Shape'], SH.targetNode, flow_meter.node))
shapes_graph.add((tc.PH_SHAPES_NREL['NREL-Natural-Gas-Meter-Shape'], SH.targetNode, thermal_meter.node))
conforms, results_graph, results = validate(hg, shacl_graph=shapes_graph, ont_graph=h_ont)

In [9]:
if conforms:   
    print(conforms, results_graph, results)
    with open(ttl_path, 'w') as file:
        file.write(ttl_data)   

True [a rdfg:Graph;rdflib:storage [a rdflib:Store;rdfs:label 'Memory2']]. Validation Report
Conforms: True

